In [4]:
import cv2
import numpy as np
import time

# 3x3 SYSTOLIC CONVOLUTION 
def systolic_conv3x3(img, kernel):
    img = img.astype(np.float32)
    H, W = img.shape

    # Padding
    padded = cv2.copyMakeBorder(img, 1, 1, 1, 1, cv2.BORDER_REFLECT)

    # Output
    out = np.zeros_like(img)

    # Systolic shift window buffers
    w0 = np.zeros((H, W), np.float32)
    w1 = np.zeros((H, W), np.float32)
    w2 = np.zeros((H, W), np.float32)

    # Load systolic rows (pipeline)
    w0[:, :] = padded[0:H,     0:W]
    w1[:, :] = padded[1:H+1,   0:W]
    w2[:, :] = padded[2:H+2,   0:W]

    # 3×3 convolution 
    out = (
        w0[:-2, :-2] * kernel[0,0] + w0[:-2,1:-1] * kernel[0,1] + w0[:-2,2:] * kernel[0,2] +
        w1[1:-1, :-2] * kernel[1,0] + w1[1:-1,1:-1] * kernel[1,1] + w1[1:-1,2:] * kernel[1,2] +
        w2[2:, :-2] * kernel[2,0] + w2[2:,1:-1] * kernel[2,1] + w2[2:,2:] * kernel[2,2]
    )
    return out

# SHARPENING KERNEL 
kernel = np.array([
    [0, -1,  0],
    [-1, 5, -1],
    [0, -1,  0]
], dtype=np.float32)

# VIDEO CAPTURE + SYSTOLIC CONVOLUTION PROCESSING
cap = cv2.VideoCapture(0)

prev = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Calculate FPS
    now = time.time()
    fps = 1 / (now - prev)
    prev = now

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # REAL-TIME SYSTOLIC 3×3 CONVOLUTION 
    conv = systolic_conv3x3(gray, kernel)

    # Normalize + convert
    conv_norm = cv2.normalize(conv, None, 0, 255, cv2.NORM_MINMAX)
    conv_uint8 = conv_norm.astype(np.uint8)

    cv2.putText(frame, f"FPS: {fps:.1f}", (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("Original", frame)
    cv2.imshow("Systolic Accurate Output", conv_uint8)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()